In [1]:
import torch
import os
import torchvision.utils as vutils
import torch.nn as nn
import torch.nn.functional as F
from models import weights_init, Discriminator, Generator
from operation import copy_G_params, load_params
from diffaug import DiffAugment
policy = 'color,translation'



In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(torch.device("cuda"))

model_path = os.path.abspath("models\\all_40000.pth")  # 確保這個檔案存在

if not os.path.exists(model_path):
    raise FileNotFoundError(f"模型權重檔案不存在: {model_path}")


# latent_dim = 100  # 你的 Generator 使用的是 nz=100
netG = Generator(im_size=256, nz=256)
netG.apply(weights_init)

netD = Discriminator(im_size=256)
netD.apply(weights_init)

netG.to(device)
netD.to(device)

ckpt = torch.load(model_path, map_location=device)
netG.load_state_dict(ckpt['g'])
netD.load_state_dict(ckpt['d'])
avg_param_G = ckpt['g_ema']
load_params(netG, avg_param_G)

# netG = nn.DataParallel(netG, device_ids=[0, 1])
# netD = nn.DataParallel(netD, device_ids=[0, 1])



cuda


C:\Users\b4088\AppData\Local\Temp\ipykernel_31140\2204126162.py:20: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(model_path, map_location=device)


In [3]:
# list all layer
for name, param in netG.named_children():
    print(name)

init
feat_8
feat_16
feat_32
feat_64
feat_128
feat_256
se_64
se_128
se_256
to_128
to_big


Freeze specific layers

In [4]:
import torch.optim as optim

# targets = [ 'se_64', 'se_128', 'se_256', 'to_big']
targets = ['init', 'feat_8', 'feat_16', 'se_64', 'se_128', 'se_256', 'to_big']
for name, layer in netG.named_children():
    for param in layer.parameters():
        if(name in targets):
            param.requires_grad = False
        else:
            param.requires_grad = True

nlr = 0.0002
nbeta1 = 0.0
weight_decay = 0.0003
g_optimizer = optim.Adam(netG.parameters(), lr=nlr, betas=(nbeta1, 0.99), weight_decay=weight_decay)
d_optimizer = optim.Adam(netD.parameters(), lr=nlr, betas=(nbeta1, 0.99), weight_decay=weight_decay)

Load data

In [5]:
from operation import ImageFolder
import torchvision.transforms as transforms
from torch.utils.data.dataloader import DataLoader
from operation import ImageFolder, InfiniteSamplerWrapper


# data_root = 'C:\\Users\\b4088\\OneDrive\\Desktop\\DeepLearning\\Assignment\\FinalProject\\few-shot-image-datasets\\few-shot-images\\art-painting\\img'
data_root = 'style_augmentation'
transform_list = [
            transforms.Resize((int(256),int(256))),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
        ]
trans = transforms.Compose(transform_list)

dataset = ImageFolder(root=data_root, transform=trans)
print(len(dataset))
dataloader = iter(DataLoader(dataset, batch_size=4, shuffle=False,
                      sampler=InfiniteSamplerWrapper(dataset), num_workers=2, pin_memory=True))


19


Model learn new style

In [ ]:
from tqdm import tqdm
from train import train_d
from diffaug import DiffAugment
policy = 'color,translation'

# set iteration you want
# a = [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
a = [50]
for j in range (0, len(a)):
    for i in tqdm(range(0, a[j])):
        real_images = next(dataloader)
        real_images = real_images.to(device)
        real_images = DiffAugment(real_images, policy=policy)

        batch_size = real_images.size(0)

        z = torch.randn(batch_size, 256, 1, 1).to(device)

        fake_images = netG(z)
        fake_images = [DiffAugment(fake, policy=policy) for fake in fake_images]


        # train disc
        netD.zero_grad()

        err_dr, rec_img_all, rec_img_small, rec_img_part = train_d(netD, real_images, label="real")
        train_d(netD, [fi.detach() for fi in fake_images], label="fake")
        d_optimizer.step()

        # train gen
        netG.zero_grad()
        pred_g = netD(fake_images, "fake")
        err_g = -pred_g.mean()

        err_g.backward()
        g_optimizer.step()


    torch.save({'g': netG.state_dict(),'g_ema': netG.state_dict(), 'd': netD.state_dict()}, f"trans_models/iter{a[j]}_augmentation.pth")


Setting up [LPIPS] perceptual loss: trunk [vgg], v[0.1], spatial [off]


c:\Users\b4088\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\b4088\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Loading model from: c:\Users\b4088\AppData\Local\Programs\Python\Python312\Lib\site-packages\lpips\weights\v0.1\vgg.pth


c:\Users\b4088\AppData\Local\Programs\Python\Python312\Lib\site-packages\lpips\lpips.py:107: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.load_state_dict(torch.load(mo

generate images

In [ ]:
import torch
import os
import torchvision.utils as vutils
import torch.nn as nn
import torch.nn.functional as F
from models import weights_init, Discriminator, Generator
from operation import copy_G_params, load_params


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_path = os.path.abspath("models/all_40000.pth") 
# mode2_path = os.path.abspath(f"trans_models/iter50.pth") 
# mode3_path = os.path.abspath(f"trans_models/iter50_augmentation.pth") 
if not os.path.exists(model_path):
    raise FileNotFoundError(f"模型權重檔案不存在: {model_path}")


netG = Generator(im_size=256, nz=256)
netG.apply(weights_init)
netG.to(device)
ckpt = torch.load(model_path, map_location=device)
netG.load_state_dict(ckpt['g'])


# netGL = Generator(im_size=256, nz=256)
# netGL.apply(weights_init)
# netGL.to(device)
# ckptL = torch.load(mode2_path, map_location=device)
# netGL.load_state_dict(ckptL['g'])

# netGLA = Generator(im_size=256, nz=256)
# netGLA.apply(weights_init)
# netGLA.to(device)
# ckptLA = torch.load(mode3_path, map_location=device)
# netGLA.load_state_dict(ckptLA['g'])

num_samples = 4  # 產生 4 張圖片
z = torch.randn(num_samples, 256).to(device)  


with torch.no_grad():
    generated_images = netG(z)[0]

generated_images = (generated_images + 1) / 2  # 轉換到 [0,1] 範圍
for i, img in enumerate(generated_images):
    vutils.save_image(img, f"result/generated_new110_{i}.png")
    # vutils.save_image(img, f"GAN/generated_{i}.png")

print(f"✅ iter100_圖片已生成並儲存！")


C:\Users\b4088\AppData\Local\Temp\ipykernel_22600\3149870624.py:28: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(model_path, map_location=device)


✅ iter100_圖片已生成並儲存！
